In [47]:
print('BEGIN')

import os
import shutil
import glob
import sys
import inspect
import filecmp

CRED = '\033[91m'
CEND = '\033[0m'

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

workdir='/OSM/CBR/OA_DCFP/work/col414/cafepp' #location of working code to compare with...
topdir='/OSM/CBR/OA_DCFP/apps/col414'         #location of clean code to compare with

os.chdir(topdir)

print('Current Working Directory=',os.getcwd())

if(not os.path.exists(topdir+'/'+'rsync_stage')):
  os.mkdir(topdir+'/'+'rsync_stage')

clean_cafepp_directory=topdir+'/'+'rsync_stage'+'/'+'cafepp_clean'

if(os.path.exists(clean_cafepp_directory)):
  print(clean_cafepp_directory,' file exists, need to move it to a unique directory...')
  
  number_searches=100 #choose big number later e.g. 1000

  for n in range(number_searches):

    backup_clean_cafepp_directory=topdir+'/'+'rsync_stage'+'/'+'cafepp_clean.old'+str(n)
    print('n,backup_clean_cafepp_directory=',n,backup_clean_cafepp_directory)
    if(not os.path.exists(backup_clean_cafepp_directory)):
      print('Found unique backup_clean_cafepp_directory=',backup_clean_cafepp_directory)
      break
  
  print('Move '+clean_cafepp_directory+' to '+backup_clean_cafepp_directory)

  shutil.move(clean_cafepp_directory,backup_clean_cafepp_directory)
  
os.chdir(topdir+'/'+'rsync_stage')

print('Current Working Directory=',os.getcwd())

print('Running git...')

os.system('git clone https://github.com/markcollier/cafepp.git')

shutil.move(topdir+'/'+'rsync_stage'+'/'+'cafepp',clean_cafepp_directory)

suffixes=['py','ipynb','json','bash','csh']

#top directory:
for suffix_cnt,suffix in enumerate(suffixes):
  files=glob.glob(clean_cafepp_directory+'/'+'*.'+suffix)
  #print('files=',files)
  for file in files:
    head,tail=os.path.split(file)
    #print(head,tail)
    
    if(not os.path.exists(workdir+'/'+tail)):
      print(CRED+workdir+'/'+tail+' missing from source directory...'+CEND)
      
    if(filecmp.cmp(file,workdir+'/'+tail)):
      print(file +' and '+workdir+'/'+tail+' same.')
    else:
      print(CRED+file +' and '+workdir+'/'+tail+' different.'+CEND)
      
#sub-directory
sub_directory='JsonTemplates'
for suffix_cnt,suffix in enumerate(suffixes):
  files=glob.glob(clean_cafepp_directory+'/'+sub_directory+'/'+'*.'+suffix)
  for file in files:
    head,tail=os.path.split(file)
    if(not os.path.exists(workdir+'/'+sub_directory+'/'+tail)):
      print(CRED+workdir+'/'+sub_directory+'/'+tail+' missing from source directory...'+CEND)
    if(filecmp.cmp(file,workdir+'/'+sub_directory+'/'+tail)):
      print(file +' and '+workdir+'/'+tail+' same.')
    else:
      print(CRED+file +' and '+workdir+'/'+sub_directory+'/'+tail+' different.'+CEND)

#sub-directory
sub_directory='TablesTemplates'
for suffix_cnt,suffix in enumerate(suffixes):
  files=glob.glob(clean_cafepp_directory+'/'+sub_directory+'/'+'*.'+suffix)
  for file in files:
    head,tail=os.path.split(file)
    if(not os.path.exists(workdir+'/'+sub_directory+'/'+tail)):
      print(CRED+workdir+'/'+sub_directory+'/'+tail+' missing from source directory...'+CEND)
    if(filecmp.cmp(file,workdir+'/'+sub_directory+'/'+tail)):
      print(file +' and '+workdir+'/'+tail+' same.')
    else:
      print(CRED+file +' and '+workdir+'/'+sub_directory+'/'+tail+' different.'+CEND)
      
#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
print('END')

BEGIN
Current Working Directory= /OSM/CBR/OA_DCFP/apps/col414
/OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean  file exists, need to move it to a unique directory...
n,backup_clean_cafepp_directory= 0 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old0
n,backup_clean_cafepp_directory= 1 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old1
n,backup_clean_cafepp_directory= 2 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old2
n,backup_clean_cafepp_directory= 3 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old3
n,backup_clean_cafepp_directory= 4 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old4
n,backup_clean_cafepp_directory= 5 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old5
n,backup_clean_cafepp_directory= 6 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old6
n,backup_clean_cafepp_directory= 7 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/cafepp_clean.old7
n,backup_clean_cafepp_directory= 8 /OSM/CBR/OA_DCFP/apps/col414/rsync_stage/c